In [1]:
from pandas import DataFrame, Series
import pandas as pd; import numpy as np

In [2]:
#Get data only for users who have posted reviews in categories Chinese^(restaurant v food)
import json
USERPATH='../full_dataset/user.json'
FS1PATH='../full_dataset/featureset1_resto_food_chinese.json'
userframe = DataFrame([json.loads(line) for line in open(USERPATH)])[['user_id', 'friends']]
active_users_frame = pd.read_json(FS1PATH)
userframe = pd.merge(userframe, active_users_frame, on='user_id')[['user_id', 'friends']]
active_users_frame = None
userframe.head() #Keep users w/out friends: they may still post reviews, which we want to count.

,user_id,friends
0,UTS9XcT14H2ZscRIf0MYHQ,"[rpOyqD_893cqmDAtJLbdog, 4U9kSBLuBDU391x6bxU-Y..."
1,ZWOj6LmzwGvMDh-A85EOtA,"[XejvgD4CrkR7FeIkjwAjtQ, T7t5UdadpHua-YnmiFpcZ..."
2,za5Q2uYrTmky9w_3h9k_wQ,"[23bCUONbJfIrtsLgX7Dgvg, l1eXX3p2WL_02FWw5TXhs..."
3,AzoZZ1hRToPsnUPXeRXelg,"[J3rNWRLRuZJ_0xsJalIhlA, Mx-vxv_V-SQCe76w4RmUfA]"
4,rYI9SVnGuPcWyhr1RiZgaw,[]


In [3]:
#Flatten "friends" field, merge back into user-friend matrix
flatteningframe = DataFrame([[i, friend]
                                     for i, friends in userframe['friends'].apply(list).iteritems()
                                         for friend in friends],
                                columns=['index', 'friend_id'])
flatteningframe = flatteningframe.set_index('index')
graph_base_frame = pd.merge(userframe, flatteningframe, left_index = True, right_index = True)[['user_id', 'friend_id']]
graph_base_frame.head()

,user_id,friend_id
0,UTS9XcT14H2ZscRIf0MYHQ,rpOyqD_893cqmDAtJLbdog
0,UTS9XcT14H2ZscRIf0MYHQ,4U9kSBLuBDU391x6bxU-YA
0,UTS9XcT14H2ZscRIf0MYHQ,Re447krbp0VQVX3Dk4Tsog
0,UTS9XcT14H2ZscRIf0MYHQ,kJc9YBRwmmZ_PG0uLHuEPQ
0,UTS9XcT14H2ZscRIf0MYHQ,Rir-YRPPClKXDFQbc3BsVw


In [4]:
flatteningframe = None
len(graph_base_frame.index)

845530

In [5]:
#TODO: Select only authorities for this step
elite_users = pd.read_json('../full_dataset/elite_users.json')
user_user = DataFrame(pd.merge(elite_users, graph_base_frame, left_on='user_id', right_on='user_id')[['user_id', 'friend_id']])
user_user['distance'] = 1
elite_users = None
len(user_user.index)

4429

In [6]:
currentGraph = DataFrame(pd.merge(user_user, graph_base_frame, left_on='friend_id', right_on='user_id')[['user_id_x', 'friend_id_y']])
currentGraph['distance'] = 2
currentGraph.columns = ['user_id', 'friend_id', 'distance']
user_user = pd.concat([user_user, currentGraph])
user_user = user_user.drop_duplicates(subset=['user_id', 'friend_id'], keep='first')
currentGraph = user_user.loc[user_user['distance'] == 2][['user_id', 'friend_id']]
len(currentGraph.index)

297366

In [7]:
currentGraph = DataFrame(pd.merge(currentGraph, graph_base_frame, left_on='friend_id', right_on='user_id')[['user_id_x', 'friend_id_y']])
currentGraph['distance'] = 3
currentGraph.columns = ['user_id', 'friend_id', 'distance']
user_user = pd.concat([user_user, currentGraph])
user_user = user_user.drop_duplicates(subset=['user_id', 'friend_id'], keep='first')
currentGraph = user_user.loc[user_user['distance'] == 3][['user_id', 'friend_id']]
len(currentGraph.index)

2055174

In [8]:
currentGraph = DataFrame(pd.merge(currentGraph, graph_base_frame, left_on='friend_id', right_on='user_id')[['user_id_x', 'friend_id_y']])
currentGraph['distance'] = 4
currentGraph.columns = ['user_id', 'friend_id', 'distance']
user_user = pd.concat([user_user, currentGraph])
user_user = user_user.drop_duplicates(subset=['user_id', 'friend_id'], keep='first')
currentGraph = user_user.loc[user_user['distance'] == 4][['user_id', 'friend_id']]
len(currentGraph.index)

2085232

In [9]:
currentGraph = DataFrame(pd.merge(currentGraph, graph_base_frame, left_on='friend_id', right_on='user_id')[['user_id_x', 'friend_id_y']])
currentGraph['distance'] = 5
currentGraph.columns = ['user_id', 'friend_id', 'distance']
user_user = pd.concat([user_user, currentGraph])
user_user = user_user.drop_duplicates(subset=['user_id', 'friend_id'], keep='first')
currentGraph = user_user.loc[user_user['distance'] == 5][['user_id', 'friend_id']]
len(currentGraph.index)

549613

In [11]:
user_user = user_user.reset_index(drop=True)
user_user.to_json('../full_dataset/user_distance_matrix.json')